# エージェントにメモリを追加する方法

このnotebookでは、エージェントにメモリを追加する方法について説明します。このnotebookを進める前に、以下のnotebookを実行してください。これらのnotebookは、このnotebookで説明する内容につながるためです。

- [LLMチェーンにメモリを追加する](adding_memory.ipynb)
- [カスタムエージェント](../../agents/examples/custom_agent.ipynb)

エージェントにメモリを追加するために、以下の手順を行います。

1. メモリを持つLLMChainを作成します。
2. そのLLMChainを使用して、カスタムエージェントを作成します。

この演習の目的のために、検索ツールにアクセスでき、`ConversationBufferMemory`クラスを利用するシンプルなカスタムエージェントを作成します。

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSearchAPIWrapper

In [ ]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

PromptTemplate内で使用される `chat_history` 変数は、ConversationBufferMemory内のダイナミックなキー名と一致することに注意してください。

In [ ]:
from datetime import datetime
today = datetime.today().strftime("%Y年%m月%d日")
now = datetime.now().strftime("%H時%M分")
prefix = """今は{today}の{now}です。会話をして、質問に可能な限り回答してください。以下のツールにアクセスすることができます。検索をするさいのクエリーは英語と日本語の両方を使用してより確かな情報になるようにして下さい:""".format(today=today,now=now)
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix,
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0.5, model_name="gpt-4",max_tokens=2000)

これで、Memoryオブジェクトを持つLLMChainを構築し、エージェントを作成することができます。

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_chain.run(input="PC版、家庭用ゲーム機版のDead by Daylightの最新のアップデートはいつ行われましたか？")

このエージェントのメモリをテストするために、前のやりとりの情報に依存して正しく回答されるようなフォローアップの質問をすることができます。

In [ ]:
agent_chain.run(input="2023年の5月に入ってからも行われています。どのようなアップデートだったか詳細に全て説明してください。")

In [ ]:
memory.chat_memory

このエージェントは、前の質問がカナダについてだったことを記憶していて、Google検索にカナダの国歌の名前をちゃんと聞いていることがわかります。

面白いので、記憶力がないエージェントと比較してみましょう。

In [ ]:
prefix = """人と会話をして、次の質問に可能な限り回答してください。以下のツールにアクセスすることができます:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_without_memory = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_without_memory.run("PC版、家庭用ゲーム機版のDead by Daylightの最新のアップデートはいつ行われましたか？")

In [ ]:
agent_without_memory.run("2023年の5月に入ってからも行われています。どのようなアップデートだったか詳細に全て説明してください。")